## USED TOOLS:

DBScan + TfidfVectorizer

## CLUSTERING:

### IMPORT MODULES:

In [1]:
import os
import re
import csv
import nltk
import mpld3
import codecs
import collections

import math as mh
import numpy as np
import pandas as pd
import string as st
import networkx as nx
import matplotlib as mpl
import matplotlib.pyplot as plt

from time import time
from itertools import chain
from itertools import islice
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn import feature_extraction
from sklearn.cluster import MiniBatchKMeans
from nltk.stem.snowball import SnowballStemmer
from sklearn.cluster import AgglomerativeClustering
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /home/deniz/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/deniz/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

### READ CSVS:

In [2]:
with open("../data/vkusvill_items.csv", 'r', encoding = 'utf-8') as file:
    reader = csv.DictReader(file)
    catalog_df = pd.DataFrame(reader)

catalog_df = catalog_df.rename(columns = {"": "shop_id", "item_composition_txt": "item_composition", "nutrion_value_txt" : "nutrion_value"})
catalog = catalog_df.drop(["shop_id", "price", "vat", "measure_unit", "measure_value", "protein_value", "fat_value", "carb_value", 
                           "energy_value", "measure_quantum"], axis = 1)

In [3]:
categories_df = pd.read_csv("../data/vkusvill_categories.csv")
categories_df = categories_df.rename( columns = {"Unnamed: 0": "shop_id"})

### PREPARE DATA:

In [4]:
def check_categorie(df, categorie):
    result = pd.DataFrame();
    
    for line in range(len(df.index)):
        if str(categorie) in df["categories_array"][line]:
            df_string = pd.DataFrame({"item_id": [df["item_id"][line]], "item_name": [df["item_name"][line]]})
            result = pd.concat([result, df_string], ignore_index=True)

    return result
    
result = check_categorie(catalog, 10068)  # Овощи

In [5]:
result

,item_id,item_name
0,41728,Картофель молодой Египет
1,43208,"Томаты черри сладкая гроздь, 250 г"
2,44309,Капуста белокочанная резаная \Свежая капуста\
3,44310,Морковь резаная \Сочная морковь\
4,44311,Свекла резаная \Сладкая свекла\
...,...,...
144,31268,"Имбирь, 200 г"
145,21494,Перец оранжевый сладкий
146,75879,"Горох стручковый сладкий, 250 г"
147,60005,"Перец оранжевый сладкий, 450 г"


### PARS ITEM NAMES:

In [6]:
# Add more

replace_items = {
                # Предлоги
                ' в ': ' ', ' к ': ' ', " до ": ' ', " по ": ' ', " через ": ' ', " после ": ' ', " в течение ": ' ', 
                 " в продолжение ": ' ', " в заключение ": ' ', " из-за ": ' ', " над ": ' ', " под ": ' ', " перед ": ' ', 
                ' у ': ' ', " возле ": ' ', " мимо ": ' ', " около ": ' ', " от ": ' ', " ради ": ' ', " благодаря ": ' ', 
                " в силу ": ' ', " ввиду ": ' ', " вследствие ": ' ', " для ": ' ',  " на ": ' ', " в целях ": ' ', 
                " с целью ": ' ', " вопреки ": ' ', " несмотря на ": ' ', ' о ': ' ', " об ": ' ', " обо ": ' ', 
                " про ": ' ', " насчёт ": ' ', ' с ': ' ', " вроде ": ' ', " наподобие ": ' ', " как ": ' ', " без ": ' ',
                # Союзы
                 " что ": ' ', " когда ": ' ', " ибо ": ' ', " пока ": ' ', " будто ": ' ', " словно ": ' ', " если ": ' ', " кто ": ' ', 
                 " который ": ' ', " какой ": ' ', " где ": ' ', " куда ": ' ', " откуда ": ' ', " потому что ": ' ', " оттого что ": ' ', 
                 " так как ": ' ', " так что ": ' ', " лишь только ": ' ', " как будто ": ' ', " с тех пор как ": ' ', 
                 " в связи с тем что ": ' ', " для того чтобы ": ' ', ' и ': ' ', " да ": ' ', " ни ни ": ' ', " тоже ": ' ', 
                 " также ": ' ', " не только ": ' ', " не столько ": ' ', " не то чтобы ": ' ', " или ": ' ', " лишь ": ' ', 
                 " либо ": ' ', " то ": ' ', " не то ": ' ', " а ": ' ', " но ": ' ', " зато ": ' ', " однако ": ' ', " то есть ": ' ', 
                 " а именно ": ' ', " чтоб ": ' ', " чтобы ": ' ', " с тех пор ": ' ', " едва ": ' ', " прежде чем ": ' ', 
                 " перед тем как ": ' ', " точно ": ' ', " подобно ": ' ', " дабы ": ' ', " коли ": ' ', " ежели ": ' ', " раз ": ' ', 
                 " пускай ": ' ', " хотя ": ' ', " поэтому ": ' ',
                 # Другие слова
                 " шт ": ' ', " мл ": ' ', " мг ": ' ', " премиум ": ' ', " вес ": ' ', " из ": ' ', " спб ": ' ',
                 " вкусвилл ": ' ', " стандарту ": ' ', " вв ": ' ', " ве ": ' ', ' г ': ' ', ' гр ': ' ', " молодой ": ' ',
                 " сладкий ": ' ', " мини ": ' ',
                 }

In [7]:
def delete_adjectives(str):
    lst = str.split()
    answer_lst = []
    
    for i in lst:
        if not i[-2:] in ["ая", "яя", "ое", "ее", "ие", "ые", "им", "ым", "ую", "юю", "ой", "ей", "ый", "ий"]:
            answer_lst.append(i)

    answer_lst = " ".join(answer_lst)
    return answer_lst

In [8]:
def replace_all(text, replace_items):
    for i, j in replace_items.items():
        text = text.lower().replace(i, j)
        
    return text

In [9]:
def clear_item_name(item_name, replace_items):
    answer = ""
    
    for symbol in item_name:
        if symbol == '_':
            answer += ' '  
        elif symbol in [
            '\\', 
            '(', 
            ')', 
            '.', 
            ',', 
            '%'
        ] or symbol.isdigit():
            answer += ' '
        else:
            answer += symbol

    answer = answer.lower() + ' '
    
    while answer != replace_all(answer, replace_items):
        answer = replace_all(answer, replace_items)
    
    while "  " in answer:
        answer = answer.replace("  ", ' ')

    return answer.lower()

In [10]:
prepare_catalog_df = pd.Series()

for i in range(len(result.index)):
    tmp = result["item_name"][i]
    
    while tmp != clear_item_name(result["item_name"][i], replace_items):
        tmp = clear_item_name(result["item_name"][i], replace_items)
    
    prepare_catalog_df = pd.concat([prepare_catalog_df, pd.Series(tmp[:-1])], ignore_index=True)

In [11]:
for i in range(len(prepare_catalog_df.index)):
    prepare_catalog_df[i] = delete_adjectives(prepare_catalog_df[i])

In [12]:
prepare_catalog_df

0         картофель египет
1      томаты черри гроздь
2          капуста капуста
3          морковь морковь
4            свекла свекла
              ...         
144                 имбирь
145                  перец
146                  горох
147                  перец
148          перец палермо
Length: 149, dtype: object

In [13]:
final_result = pd.concat([result, prepare_catalog_df], ignore_index=True, axis=1)
final_result = final_result.rename( columns = {0: "item_id", 1: "item_name", 2: "clear_item_name"})

In [14]:
final_result

,item_id,item_name,clear_item_name
0,41728,Картофель молодой Египет,картофель египет
1,43208,"Томаты черри сладкая гроздь, 250 г",томаты черри гроздь
2,44309,Капуста белокочанная резаная \Свежая капуста\,капуста капуста
3,44310,Морковь резаная \Сочная морковь\,морковь морковь
4,44311,Свекла резаная \Сладкая свекла\,свекла свекла
...,...,...,...
144,31268,"Имбирь, 200 г",имбирь
145,21494,Перец оранжевый сладкий,перец
146,75879,"Горох стручковый сладкий, 250 г",горох
147,60005,"Перец оранжевый сладкий, 450 г",перец


### CLUSTERING:

In [15]:
stemmer = SnowballStemmer("russian")

def token_and_stem(text):
    tokens = [word for sentence in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sentence)]
    filtered_tokens = []
    
    for token in tokens:
        if re.search('[а-яА-Я]', token):
            filtered_tokens.append(token)
    
    stems = [stemmer.stem(token) for token in filtered_tokens]
    return stems

def token_only(text):
    tokens = [word.lower() for sentence in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sentence)]
    filtered_tokens = []
    
    for token in tokens:
        if re.search('[а-яА-Я]', token):
            filtered_tokens.append(token)
    
    return filtered_tokens

# Создаем словари из полученных основ
total_vocab_stem = []
total_vocab_token = []

for i in range(len(result.index)):
    all_words_stemmed = token_and_stem(final_result["clear_item_name"][i])
    # print(all_words_stemmed)
    total_vocab_stem.extend(all_words_stemmed)    
    all_words_tokenized = token_only(final_result["clear_item_name"][i])
    total_vocab_token.extend(all_words_tokenized)

In [16]:
print('Товаров из категории считано: ' + str(len(final_result)))

Товаров из категории считано: 149


In [17]:
#можно расширить список стоп-слов

stopwords = nltk.corpus.stopwords.words('russian')
stopwords.extend(['что', 'это', 'так', 'вот', 'быть', 'как', 'в', 'к', 'на', "из", "по", 'г', "вв", "вкуссвилл", "вес", "спб", ""])

# ["были" том пожалуйста около вон день недалеко самими сам мож кто много восемнадцатый к же четыре ваше близко им несколько мира ту по, "быть", 
# "которой", "тринадцать "потом "никуда "свое "говорит "одиннадцать ваша "можно "этом "туда", "само", "говорил твоя будешь разве наверху 
# этой ней тысяч твоё саму двенадцатый всё нем перед время весь обычно позже второй ещё пятый чего имя вокруг другая чаще сказала та самих
# многочисленная раньше отсюда всею самим моё между тобой вдруг у есть слишком про нет эту какая пять всего тринадцатый свои их 
# когда ними мои более меля нередко нас себя совсем сих этих меньше уж сказал жизнь кроме самой всю её кем ты и одиннадцатый три
# ни суть двадцатый десятый году пятнадцатый такие будете которых самого чтобы нам часто чтоб конечно два зачем всему многочисленные
# будут их они наше пора нибудь заняты оба могут всюду очень каждая лишь рядом твой только это все шестой люди мой самом ж внизу девятый
# себе меня почему его мимо вверх будет всегда как об которого вот собой то уже в мор мочь сказать эти снова многочисленное человек
# сколько своих она занята важный мы вы восемь чем неё двадцать дальше ком них семнадцать лучше отовсюду всем седьмой других он который
# сегодня надо такое оно больше моя нею вниз вами другое шестнадцатый сами мог теми такая нужно иметь этого многочисленный после однажды
# теперь наиболее занято времени
# самому года посреди потому шесть тоже начала тех хотя один ниже каждый может каждые сейчас собою девять мало наши важные 
# где долго также которая тебе было чуть куда рано занят действительно для вам впрочем что всеми четырнадцать без нельзя против 
# а тебя почти прекрасно вся семнадцатый друго мною
# своего сначала процентов стал даже дел те эта кому миллионов затем вас ли значит давно первый пор ее ваши раз тут какой недавно уметь сеаой 
# немного наконец именно довольно ним т зато хотеть важная до иногда ему тот две кругом мне которые просто или там пока менее бывь опять десять
# вдали хоть ими лет другой восьмой хочешь еще наша ведь тому через был здесь сама во этим двенадцать бывает нему этими него
# буду кажется была пятнадцать не с можхо наш ей спасибо другие тобою хорошо двух из
# четырнадцатый этот одного низко важное даром будь ну ею
# "однако", "будем", "мной", "должно", "тою", "нами", "девятнадцатый", "ваш на
# нее всех алло тем далеко своей так особенно восемнадцать ничего над четвертый
# кого шестнадцать год такой чему под если будто того одной свою везде при но со", 'о', "от", "да", "семь", 'г', "за", "никогда", "непрерывно", 
# "этому", "каждое", "третий", "девятнадцать", "назад", "тогда", "бы"]

n_featur = 20000
tfidf_vectorizer = TfidfVectorizer(max_features=10000,
                                 min_df = 1, stop_words=stopwords,
                                 use_idf=True, tokenizer=token_and_stem, ngram_range=(1, 1))
get_ipython().magic('time tfidf_matrix = tfidf_vectorizer.fit_transform(final_result["clear_item_name"])')
print(tfidf_matrix.shape)

CPU times: user 52.1 ms, sys: 69 µs, total: 52.1 ms
Wall time: 51.7 ms
(149, 67)


/tmp/ipykernel_37852/2051668285.py:34: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  get_ipython().magic('time tfidf_matrix = tfidf_vectorizer.fit_transform(final_result["clear_item_name"])')
/home/deniz/.local/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/home/deniz/.local/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:408: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['бол', 'больш', 'будт', 'быт', 'вед', 'впроч', 'всег', 'всегд', 'даж', 'друг', 'е', 'ег', 'ем', 'есл', 'ест', 'ещ', 'зач', 'зде', 'ил', 'иногд', 'когд', 'конечн', 'куд', 'лучш', 'межд', 'мен', 'мног', 'мо', 'можн', 'нег', 'нельз', 'нибуд', 'никогд', 'нич', 'опя', 'посл', 'пот', 'почт', 'разв', 'сво', 'себ', 'совс'

In [18]:
get_ipython().magic('time db = DBSCAN(eps=0.0001, min_samples=1).fit(tfidf_matrix)')
labels = db.labels_
labels.shape

CPU times: user 24 ms, sys: 0 ns, total: 24 ms
Wall time: 20.9 ms


/tmp/ipykernel_37852/3451584040.py:1: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  get_ipython().magic('time db = DBSCAN(eps=0.0001, min_samples=1).fit(tfidf_matrix)')


(149,)

In [19]:
print(labels)
final_result["clear_item_name"]

[ 0  1  2  3  4  2  5  2  6  7  8  9 10 11  1 12 13  8 14  7  4 15 16 17
 18 19 20 21 22 23 24 10 25 26 20 27 27 28 29 30 31 32 12 14 33 27 34 27
 35 36 30 37 38 39 40 41 42 43 44 45 46 47 48 49 50  2 51  3  8 10  4 52
 25  2 53 54 55 18  2 56 29 57 57 29 58 10 10  3 59  8 60 61  3 22 10 57
 62 63 10 19  9 64 51  4  8  8 65 30 66 67 27 10 51 68 27 69 58 70 18 27
 51 51 71 27 72 73 74 59 74 12 75 76 77 78 69 79 13 15 12 39  3 80  1 81
 82  8 83  8 81]


0         картофель египет
1      томаты черри гроздь
2          капуста капуста
3          морковь морковь
4            свекла свекла
              ...         
144                 имбирь
145                  перец
146                  горох
147                  перец
148          перец палермо
Name: clear_item_name, Length: 149, dtype: object

In [20]:
clusters3 = labels
out = { 'title': final_result["clear_item_name"], 'cluster': clusters3 }
clusters3 = pd.Series(clusters3)
final_result = pd.concat([final_result, clusters3], ignore_index = True, axis=1)

In [21]:
final = final_result.rename(columns = {0: "shop_id", 2: "item_name", 3: "cluster"})
final = final.drop(1, axis=1)
final = final.sort_values(by='cluster')

In [22]:
final[:55]

,shop_id,item_name,cluster
0,41728,картофель египет,0
1,43208,томаты черри гроздь,1
14,55555,томаты черри гроздь,1
142,38659,томаты черри гроздь,1
5,44920,капуста,2
2,44309,капуста капуста,2
73,626,капуста,2
78,14403,капуста,2
65,602,капуста,2
7,46786,капуста,2
